In [1]:
import itertools
import pathlib
import pyam

import numpy as np
import pandas as pd

c:\Users\gidden\Miniconda3\envs\pyam\Lib\site-packages\xarray\backends\cfgrib_.py:27: UserWarning: Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. Try `import cfgrib` to get the full error message
  warnings.warn(


In [2]:
all_new_meta = []
all_categories = ['C1', 'C2', 'C3']

# Load Data

In [3]:
data_path = pathlib.Path('../data')
paper_prefix = '10.5281_zenodo.10158920_gidden_et_al_2023_ar6_reanalysis'

In [4]:
paper_data = pd.concat([
    pd.read_csv(data_path / f'{paper_prefix}_data.csv'),
    pd.read_csv(data_path / 'additional_analysis_data.csv'),
])

In [5]:
pyam_paper_data = pyam.IamDataFrame(paper_data)
pyam_paper_data.load_meta(data_path / f'{paper_prefix}_meta.csv')
pyam_paper_data.meta = pyam_paper_data.meta.drop(columns=['Category']) # this will come back when we load in AR6 data

[WARNING] 12:36:09 - pyam.core: No meta indicators for the following scenarios:
         model                scenario
0  AIM/CGE 2.0  ADVANCE_2020_1.5C-2100
1  AIM/CGE 2.0      ADVANCE_2020_Med2C
2  AIM/CGE 2.0       ADVANCE_2020_WB2C
3  AIM/CGE 2.0      ADVANCE_2030_Med2C
4  AIM/CGE 2.0  ADVANCE_2030_Price1.5C
...
[INFO] 12:36:09 - pyam.core: Found column 'exclude' in sheet 'meta', moved to attribute `IamDataFrame.exclude`.


In [6]:
ar6_data = pd.read_csv(data_path / 'AR6_Scenarios_Database_World_v1.0.csv')

In [7]:
variables = {
    'AR6 climate diagnostics|Infilled|Emissions|CO2': 
    'AR6 climate diagnostics|Infilled|Emissions|CO2', 
    'AR6 climate diagnostics|Native-with-Infilled|Emissions|Kyoto Gases (AR6-GWP100)': 
    'AR6 climate diagnostics|Infilled|Emissions|Kyoto Gases',
}
pyam_ar6 = pyam.IamDataFrame(ar6_data[ar6_data.Variable.isin(list(variables) + ['Emissions|CO2', 'Emissions|Kyoto Gases'])])
pyam_ar6.load_meta(data_path / 'AR6_Scenarios_Database_metadata_indicators_v1.0.xlsx')
pyam_ar6 = (
    pyam_ar6
    .filter(Category=all_categories)
    .rename(variable=variables)
)
columns = {
    'Year of netzero CO2 emissions (Harm-Infilled) table': 
    'Interpolated Net-Zero Year for AR6 climate diagnostics|Infilled|Emissions|CO2',
    'Year of netzero GHG emissions (Harm-Infilled) table': 
    'Interpolated Net-Zero Year for AR6 climate diagnostics|Infilled|Emissions|Kyoto Gases',
}
pyam_ar6.meta.rename(columns=columns, inplace=True)
pyam_ar6.meta = pyam_ar6.meta[['Category'] + list(columns.values())]

[WARNING] 12:36:15 - pyam.core: Ignoring meta indicators for the following scenarios:
         model                       scenario
0  AIM/CGE 2.2   ADVANCE_WP2_IND-450-FullTech
1  AIM/CGE 2.2  ADVANCE_WP2_IND-Base-FullTech
2  AIM/CGE 2.2   ADVANCE_WP2_TRA-450-FullTech
3  AIM/CGE 2.2  ADVANCE_WP2_TRA-Base-FullTech
4  AIM/CGE 2.2  ADVANCE_WP2_TRA-Ctax-FullTech
...


### Combine data sets

We expect to have different numbers of scenarios, since `paper_data` reports only those scenarios which can be run with OSCAR.

In [8]:
df = pyam.concat([pyam_paper_data, pyam_ar6])

In [9]:
len(df.filter(variable='Cumulative AR6 climate diagnostics|Infilled|Emissions|CO2 from 2020').filter(Category='C1').index)

97

In [10]:
len(df.filter(variable='Cumulative AR6 Reanalysis|OSCARv3.2|Emissions|CO2 from 2020').filter(Category='C1').index)

70

# Compute Carbon Budgets

In [11]:
meta_variable_templates = [
    'AR6 climate diagnostics|Infilled|Emissions|{gas}', # as reported in AR6
    'AR6 Reanalysis|OSCARv3.2|Emissions|{gas}|Direct Only', # reanalyzed by OSCAR
    'AR6 Reanalysis|OSCARv3.2|Emissions|{gas}', # reanalyzed by OSCAR including NGHGI adjustment 
]

data_variable_templates = [
    'AR6 climate diagnostics|Infilled|Emissions|{gas}', # as reported in AR6
    'AR6 Reanalysis|OSCARv3.2|Emissions|{gas}', # reanalyzed by OSCAR
    'AR6 Reanalysis|OSCARv3.2|Emissions|{gas} - Direct and Indirect Fluxes', # reanalyzed by OSCAR including NGHGI adjustment 
]

In [12]:
def make_carbon_budget(df, mvariable, dvariable):
    meta = (
        df
        .meta
        .query(f"Category in {all_categories}")
        [f'Interpolated Net-Zero Year for {mvariable}']
        .dropna()
        .astype(int)
        .reset_index()
    ) 
    data = (
        df
        .filter(region='World', variable=f'Cumulative {dvariable} from 2020')
        .convert_unit('Mt CO2/yr', to='Gt CO2', factor=1e-3)
    )
    budgets = [(
        row['model'], 
        row['scenario'], 
        data.filter(
            model=row['model'], scenario=row['scenario'], 
            year=row[f'Interpolated Net-Zero Year for {mvariable}']
        ).timeseries().values[0][0]
    )
        for i, row in meta.iterrows()
    ]
    return pd.DataFrame(budgets, columns=['model', 'scenario', f'Carbon Budget for {dvariable}']).set_index(['model', 'scenario'])
        

In [13]:
new_meta = pd.concat([
    make_carbon_budget(df, mtemplate.format(gas='CO2'), dtemplate.format(gas='CO2')) for mtemplate, dtemplate in zip(meta_variable_templates, data_variable_templates)
], axis=1)
all_new_meta.append(new_meta)
df.set_meta(new_meta)
new_meta.head()

Carbon Budget for AR6 climate diagnostics|Infilled|Emissions|CO2  \
model       scenario                                                                                  
AIM/CGE 2.0 SSP1-26                                                       945.615367                  
            SSP4-26                                                       982.207779                  
AIM/CGE 2.1 CD-LINKS_NPi2020_1000                                         679.395766                  
            CD-LINKS_NPi2020_400                                          508.452135                  
AIM/CGE 2.2 EN_INDCi2030_1000f                                           1160.102947                  

                                   Carbon Budget for AR6 Reanalysis|OSCARv3.2|Emissions|CO2  \
model       scenario                                                                          
AIM/CGE 2.0 SSP1-26                                                       862.102117          
            SSP4-26                                                      1036.223839          
AIM/CGE 2.1 CD-LINKS_NPi2020_1000                                         493.573297          
            CD-LINKS_NPi2020_400                                          364.560104          
AIM/CGE 2.2 EN_INDCi2030_1000f                                            989.933137          

                                   Carbon Budget for AR6 Reanalysis|OSCARv3.2|Emissions|CO2 - Direct and Indirect Fluxes  
model       scenario                                                                                                      
AIM/CGE 2.0 SSP1-26                                                       711.308853                                      
            SSP4-26                                                       876.356706                                      
AIM/CGE 2.1 CD-LINKS_NPi2020_1000                                         374.487241                                      
            CD-LINKS_NPi2020_400                                          298.821486                                      
AIM/CGE 2.2 EN_INDCi2030_1000f                                            820.869212

# Emission Reductions

In [14]:
def compute_reduction(df, variable, fromyear=2020, toyear=2030):
    data = (
        df
        .filter(variable=variable, year=[fromyear, toyear], region='World')
        .timeseries()
        .droplevel(['region', 'unit', 'variable'])
    )
    s = (data[fromyear] - data[toyear]) / data[fromyear] * 100
    s.name = f'2030 Emission Reductions for {variable}'
    return s

In [15]:
gases = ['CO2', 'Kyoto Gases']
new_meta = pd.concat([
    compute_reduction(df, template.format(gas=gas))
    for gas, template in itertools.product(gases, data_variable_templates)
], axis=1)
all_new_meta.append(new_meta)
df.set_meta(new_meta)
new_meta.head()

2030 Emission Reductions for AR6 climate diagnostics|Infilled|Emissions|CO2  \
model       scenario                                                                                             
AIM/CGE 2.0 SSP1-26                                                        20.698212                             
            SSP4-26                                                        23.080940                             
AIM/CGE 2.1 CD-LINKS_NPi2020_1000                                          39.680445                             
            CD-LINKS_NPi2020_400                                           44.295652                             
            CO_Bridge                                                      22.553044                             

                                   2030 Emission Reductions for AR6 Reanalysis|OSCARv3.2|Emissions|CO2  \
model       scenario                                                                                     
AIM/CGE 2.0 SSP1-26                                                        20.690203                     
            SSP4-26                                                        19.278438                     
AIM/CGE 2.1 CD-LINKS_NPi2020_1000                                          50.210320                     
            CD-LINKS_NPi2020_400                                           56.814615                     
            CO_Bridge                                                      28.227574                     

                                   2030 Emission Reductions for AR6 Reanalysis|OSCARv3.2|Emissions|CO2 - Direct and Indirect Fluxes  \
model       scenario                                                                                                                  
AIM/CGE 2.0 SSP1-26                                                        24.188745                                                  
            SSP4-26                                                        22.529429                                                  
AIM/CGE 2.1 CD-LINKS_NPi2020_1000                                          56.662343                                                  
            CD-LINKS_NPi2020_400                                           63.605357                                                  
            CO_Bridge                                                      33.335398                                                  

                                   2030 Emission Reductions for AR6 climate diagnostics|Infilled|Emissions|Kyoto Gases  \
model       scenario                                                                                                     
AIM/CGE 2.0 SSP1-26                                                        24.976649                                     
            SSP4-26                                                        28.116878                                     
AIM/CGE 2.1 CD-LINKS_NPi2020_1000                                          41.311850                                     
            CD-LINKS_NPi2020_400                                           45.320361                                     
            CO_Bridge                                                      27.958333                                     

                                   2030 Emission Reductions for AR6 Reanalysis|OSCARv3.2|Emissions|Kyoto Gases  \
model       scenario                                                                                             
AIM/CGE 2.0 SSP1-26                                                        25.684139                             
            SSP4-26                                                        26.476508                             
AIM/CGE 2.1 CD-LINKS_NPi2020_1000                                          49.059937                             
            CD-LINKS_NPi2020_400                                           54.422898                             
        

# Save to File

In [16]:
to_save = pd.concat(all_new_meta + [pyam_ar6.meta[list(columns.values())]]) # add renamed columns

In [17]:
to_save.to_csv(data_path / 'additional_analysis_metadata.csv', index=True)